# Cargamos utilidades de colab y TaLib

In [36]:
# !git clone https://github.com/b0tm4r/RNN---Divisas.git
from google.colab import drive # Para cargar el disco
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador

drive.mount('/content/drive')
!ls '/content/drive/My Drive' 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'


# Instalar Ta-lib



In [37]:
import os
# download TA-Lib 
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
!tar xvzf ta-lib-0.4.0-src.tar.gz
os.chdir('ta-lib') # co-lab no admite cd
!./configure --prefix=/usr
!make
!make install
# wait ~ 30s
os.chdir('../')
!pip install TA-Lib

--2021-01-13 17:00:44--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-01-13 17:00:44--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-01-13 17:00:45--  https://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving nchc.dl.sourceforge.ne

# Importar librerias

In [38]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import talib
import pickle

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import load_model

# Parte 1 - Carga y procesado de datos

In [39]:
## Importar el dataset de entrenamiento
def get_dataframe(dir_path, valor):
    df1 = pd.read_csv(os.path.join(dir_path,'datasets',
                                   "{}.csv".format(valor)))
    df2 = pd.read_csv(os.path.join(dir_path,'datasets',
                                   "{}-total.csv".format(valor)))
    df = pd.concat([df1, df2], ignore_index=True)
    df = df.drop_duplicates(['Fecha'])
    
    ## Datafame  df.iloc[::-1] para reverse xq los datos 
    ## vienen de mas a menos y reindexamos    
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    ## Close
    df['Close'] = df.iloc[:,1:2]
    df['Close'] = df['Close'].apply(lambda x: x.replace(',','.')).astype(float)
    ## Open
    df['Open'] = df.iloc[:,2:3]
    df['Open'] = df['Open'].apply(lambda x: x.replace(',','.')).astype(float)
    ## High
    df['High'] = df.iloc[:,3:4]
    df['High'] = df['High'].apply(lambda x: x.replace(',','.')).astype(float)
    ## Low
    df['Low'] = df.iloc[:,4:5]
    df['Low'] = df['Low'].apply(lambda x: x.replace(',','.')).astype(float)
    ## RSI 14
    rsi = talib.RSI(df['Close'])
    df['Rsi'] = rsi
    ## Selección de columnas
    df = df.iloc[:,[0,6,7,8,9,10]]
    return df

## Numero de periodos para entrenamiento  

In [40]:
dir_path = '/content/drive/My Drive/Colab Notebooks/RNN-Divisas'
periodos = 60 
valor = 'EUR-USD'

In [41]:
dataset_train = get_dataframe(dir_path,valor)
dataset_train = dataset_train.dropna()
dataset_train.head(3)

,Fecha,Close,Open,High,Low,Rsi
14,21.01.2015,1.1610,1.1553,1.1677,1.1542,19.656020
15,22.01.2015,1.1366,1.1610,1.1648,1.1317,14.859266
16,23.01.2015,1.1208,1.1366,1.1374,1.1114,12.698293


## Columnas con valores a tratar, precio y rsi


In [42]:
training_set = []
training_set.append( dataset_train.iloc[:-periodos, 1:2].values )
training_set.append( dataset_train.iloc[:-periodos, 5:].values )

## Escalado de características


In [43]:
sc = []
for i in training_set:
    sc.append(MinMaxScaler(feature_range = (0, 1)))

training_set_scaled = []
for i in range(0,len(training_set)):
    training_set_scaled.append(sc[i].fit_transform(training_set[i]))

## Crear una estructura de datos con 60 timesteps por columna y 1 salida 


In [44]:
X_train = []
y_train = []
for i in range(0,len(training_set)):
    X_train_set = []
    y_train_set = []
    for x in range(periodos, training_set[i].shape[0]):
       X_train_set.append(training_set_scaled[i][x-periodos:x, 0])
       y_train_set.append(training_set_scaled[i][x, 0])

    X_train.append(X_train_set)
    y_train.append(y_train_set)    

X_train_list = []
y_train_list = []
for i in range(0,len(training_set)):
    X_train_data, y_train_data = np.array(X_train[i]), np.array(y_train[i])
    X_train_list.append(X_train_data)
    y_train_list.append(y_train_data)

## axis = dimensiones
X_train = np.stack(X_train_list, axis=len(training_set))
## y_train se corresponde a la columna base, en este caso el precio
## xq es en aquello que estamos interesados en predecir
y_train = y_train_list[0] 

# Parte 2 - Construcción de la RNR

In [45]:
epocas = 50
batch_size = 32
units = 50
dropout = 0.2

# Inicialización del modelo
regressor = Sequential()

## Añadir la primera capa de LSTM y la regulariación por Dropout
regressor.add(LSTM(units = units, return_sequences = True,
                   input_shape = (X_train.shape[1], len(training_set)) ))
regressor.add(Dropout(dropout))

## Añadir la segunda capa de LSTM y la regulariación por Dropout
regressor.add(LSTM(units = units, return_sequences = True ))
regressor.add(Dropout(dropout))

## Añadir la tercera capa de LSTM y la regulariación por Dropout
regressor.add(LSTM(units = units, return_sequences = True ))
regressor.add(Dropout(dropout))

## Añadir la cuarta capa de LSTM y la regulariación por Dropout
regressor.add(LSTM(units = units))
regressor.add(Dropout(dropout))

## Añadir la capa de salida
regressor.add(Dense(units = 1))

## Compilar la RNR
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

## Ajustar la RNR al conjunto de entrenamiento
regressor.fit(X_train, y_train, epochs = epocas, batch_size = batch_size)

Epoch 1/50
46/46 [==============================] - 10s 99ms/step - loss: 0.0755
Epoch 2/50
46/46 [==============================] - 5s 104ms/step - loss: 0.0087
Epoch 3/50
46/46 [==============================] - 5s 103ms/step - loss: 0.0071
Epoch 4/50
46/46 [==============================] - 5s 103ms/step - loss: 0.0063
Epoch 5/50
46/46 [==============================] - 5s 103ms/step - loss: 0.0064
Epoch 6/50
46/46 [==============================] - 5s 104ms/step - loss: 0.0062
Epoch 7/50
46/46 [==============================] - 5s 103ms/step - loss: 0.0053
Epoch 8/50
46/46 [==============================] - 5s 104ms/step - loss: 0.0064
Epoch 9/50
46/46 [==============================] - 5s 103ms/step - loss: 0.0051
Epoch 10/50
46/46 [==============================] - 5s 105ms/step - loss: 0.0058
Epoch 11/50
46/46 [==============================] - 5s 105ms/step - loss: 0.0062
Epoch 12/50
46/46 [==============================] - 5s 105ms/step - loss: 0.0056
Epoch 13/50
46/46 [======

# Parte 3 -  Guardamos el modelo

In [46]:
regressor.save(os.path.join(dir_path,'models',
                            "{}-precio-rsi.h5".format(valor)))

In [47]:
pickle_file = os.path.join(dir_path,'models',
                            "{}-precio-rsi.pk".format(valor))
stringlist = []
regressor.summary(print_fn=lambda x: stringlist.append(x))

pickle.dump( {'summary' : stringlist },
            open(pickle_file,'wb'))